### Read SLHA, SModelS and micrOmegas output and store the data in a pandas DataFrame

In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
import numpy as np
import pandas as pd
import glob,imp,os,sys
from pandas import json_normalize
import pyslha
sys.path.append('../smodels')
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
from getRefXSecs import getXSecFor
from microParser import micro2Py
pd.options.mode.chained_assignment = None #Disable copy warnings

In [2]:
print(micro2Py('../EWino/data/micromegas_scanRandom/ew_zbwbb1b_.micro'))

File ../EWino/data/micromegas_scanRandom/ew_zbwbb1b_.micro not found
{}


In [3]:
model = Model(BSMparticles=BSMList, SMparticles=SMList)

In [4]:
slhaFolder = '../EWino/data/slha_scanRandom'
smodelsFolder = '../EWino/data/smodels_scanRandom'
microFolder = '../EWino/data/micromegas_scanRandom'

In [11]:
#Convert Experimental Results list to a dictionary
data = []
removeFromDict = ['topologies outside the grid',"missing topologies",
                  "missing topologies with displaced decays", 'missing topologies with prompt decays',
                 "Asymmetric Branches","Outside Grid","Missed Topologies","Long Cascades"]
for f in glob.glob(smodelsFolder+'/*.py'):
    smodelsDict = imp.load_source(f.replace('.py',''),f).smodelsOutput
    if not 'ew_c8exk4i4' in f: continue
    for rmKey in removeFromDict:
        if rmKey in smodelsDict:
            smodelsDict.pop(rmKey)
    if 'ExptRes' in smodelsDict:
        for res in smodelsDict['ExptRes']:
            if 'TxNames weights (fb)' in res:
                res.pop('TxNames weights (fb)')  
        try:
            expList = sorted(smodelsDict['ExptRes'], key=lambda pt: pt['r'],reverse=True)
        except:
            for pt in smodelsDict['ExptRes']:
                print(pt['AnalysisID'],pt['dataType'],pt['r'])
            break
        expDict = dict([['result%i'%i,val] for i,val in enumerate(expList)])
        smodelsDict['ExptRes'] = expDict
    slhaFile = smodelsDict['OutputStatus']['input file']
    dataDict = {'filename' : os.path.basename(slhaFile)}
    dataDict.update(smodelsDict)
    data.append(dataDict)

ATLAS-SUSY-2019-08 upperLimit 0.3068579
ATLAS-SUSY-2019-08 efficiencyMap 0.2219134
ATLAS-SUSY-2018-23 upperLimit 0.197469
CMS-SUS-17-004 upperLimit 0.1897095
CMS-SUS-16-043 upperLimit 0.1688579
CMS-SUS-17-004 upperLimit 0.1313516
ATLAS-SUSY-2016-24 efficiencyMap 0.09900743
ATLAS-SUSY-2016-24 upperLimit 0.08957163
CMS-SUS-16-039 upperLimit 0.07965567
CMS-SUS-16-034 upperLimit 0.07148786
CMS-SUS-16-045 upperLimit 0.063237
ATLAS-SUSY-2018-06 efficiencyMap 0.06227844
ATLAS-SUSY-2018-06 upperLimit 0.04894797
ATLAS-SUSY-2018-32 upperLimit 0.04545506
ATLAS-SUSY-2013-11 efficiencyMap 0.04307504
ATLAS-SUSY-2013-12 upperLimit 0.03919989
ATLAS-SUSY-2013-11 upperLimit 0.03489477
CMS-SUS-13-006 upperLimit 0.02667756
ATLAS-SUSY-2019-08 upperLimit 0.01133536
ATLAS-SUSY-2019-08 upperLimit 0.009333922
ATLAS-SUSY-2017-01 upperLimit 0.007936989
ATLAS-SUSY-2018-32 upperLimit 0.007757832
CMS-SUS-16-043 upperLimit 0.007200395
CMS-SUS-17-004 upperLimit 0.006774372
ATLAS-SUSY-2017-03 upperLimit 0.00637431
ATL

In [ ]:
print(len(data))

In [ ]:
#Convert data to flat DataFrame:
smodelsDF = json_normalize(data)

In [ ]:
#Get SLHA data:
slhaData = []
for f in smodelsDF['filename']:
    slhaFile = os.path.join(slhaFolder,f)
    slha = pyslha.readSLHAFile(slhaFile)
    massDict = dict([[str(key),abs(val)] for key,val in slha.blocks['MASS'].items() if key > 25])
    extparDict = dict([[str(key),val] for key,val in slha.blocks['EXTPAR'].items()])
    widthDict = dict([[str(key),val.totalwidth] for key,val in slha.decays.items() if key > 25])
    BRsDict = {}
    for pdg,val in slha.decays.items():
        if not abs(pdg) in [1000024,1000023]:
            continue
        initialState = model.getParticlesWith(pdg=pdg)[0].label            
        BRsDict[initialState] = {}
        for dec in val.decays:
            if dec.br < 0.01: continue            
            finalState = ','.join([model.getParticlesWith(pdg=pid)[0].label for pid in sorted(dec.ids)])
            BRsDict[initialState][finalState] = dec.br
    xsec8TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=8000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=8000)])
    xsec13TeV = dict([ [str(proc.pidsfinal).replace('[','').replace(']','').replace(',','_').replace(' ',''),
                   max([x.value for x in proc.get_xsecs(sqrts=13000)])*1000] 
                 for proc in slha.xsections.values()  if proc.get_xsecs(sqrts=13000)])    
    slhaDict = {'filename' : f, 'mass' : massDict, 'width' : widthDict, 'extpar' : extparDict, 
                'xsec8TeV(fb)' : xsec8TeV, 'xsec13TeV(fb)' : xsec13TeV, 'BRs' : BRsDict}
    slhaData.append(slhaDict)

In [ ]:
#Convert to DataFrame
slhaDF = json_normalize(slhaData)
#Add total cross-sections:
xsecs13 = [x for x in list(slhaDF) if 'xsec13TeV' in x]
xsecs8 = [x for x in list(slhaDF) if 'xsec8TeV' in x]
slhaDF['totalxsec13TeV(fb)'] = slhaDF[xsecs13].sum(axis=1)
slhaDF['totalxsec8TeV(fb)'] = slhaDF[xsecs8].sum(axis=1)

In [ ]:
#Merge with SModelS DataFrame
dataDF = slhaDF.merge(smodelsDF,how='inner')
# print('Final number of data points:',dataDF.shape[0])
#print(dataDF.columns.values.tolist()) #Print all columns names

In [ ]:
#Get Micromegas data:
microData = []
for f in smodelsDF['filename']:
    microFile = os.path.join(microFolder,f.replace('.slha','.micro'))
    obs_dict = micro2Py(microFile)
    obs_dict['filename'] = f
    microData.append(obs_dict)

In [ ]:
#Convert to DataFrame
microDF = json_normalize(microData)
#Merge with SModelS DataFrame
dataDF2 = microDF.merge(dataDF,how='inner')
print('Final number of data points:',dataDF2.shape[0])
#print(dataDF2.columns.values.tolist()) #Print all columns names

In [ ]:
### Add reference cross-section ratio for C1C1,C1N2:
pids = [[-1000024,1000024],[1000023,1000024],[-1000024,1000023]]
ratio = []
for index,row in dataDF2.iterrows():    
    xsecRefSum = 0.0
    xsecSum = 0.0
    for pidpair in pids:
        mass1 = row['mass.%i'%abs(pidpair[0])] 
        mass2 = row['mass.%i'%abs(pidpair[1])] 
        mass = (mass1+mass2)/2.
        xsecRef = max(0,1000*getXSecFor(pidpair[0],pidpair[1],mass,13.0,'wino'))
        xsecRefSum += xsecRef
            
        xsecLabel = 'xsec13TeV(fb).%i_%i' %(pidpair[0],pidpair[1])
        try:
            xsec = row[xsecLabel]
            if np.isnan(xsec):
                xsec = 0.0
        except:
            xsec = 0.0
        xsecSum += xsec
    if xsecRefSum > 0.0:
        ratio.append(xsecSum/xsecRefSum)    
    else:
        ratio.append(-1.0)

In [ ]:
dataDF2['xsecRatio'] = ratio

In [ ]:
#Save DataFrame to pickle file:
# dataDF.to_pickle('WinoResultsv2.pcl')

dataDF2.to_pickle('EWino_scanRandomDec.pcl')